# Import Dependencies

In [ ]:
import gensim.downloader as api
from gensim import utils
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import download
from google.colab import drive
import os

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# To Do

1. Figure out how to average document embedding - DONE
  * What do I do with out of vocabulary (OOV) words? - DONE
    * It seems like you just get rid of the OOV words. FastText does attempt to deal with this.
  * What do I do with words that appear more than once in document? - DONE
    * Average them as if they were a separate term
2. Write function to find document embedding - DONE
3. Save document embedding vectors to .npy file - DONE

# Download Embeddings

Word embeddings are 300 dimensional.

In [ ]:
wv = api.load('word2vec-google-news-300')

# Load and Pre-process Data

In [ ]:
os.chdir("/content/gdrive/MyDrive/Colab Notebooks/")

In [ ]:
download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
class MyCorpus:

  def __iter__(self):
    corpus_df = pd.read_csv('abstracts.csv')
    corpus_df.dropna(subset=['description'], inplace=True)
    corpus_df.reset_index(drop=True, inplace=True)
    for _, row in corpus_df.iterrows():
      words = utils.simple_preprocess(row['description'])
      # Remove stop words
      words = [word for word in words if word not in stop_words]
      yield words

In [ ]:
sentences = MyCorpus()

# Caculate Document Embeddings

In [ ]:
def get_doc_embedding(word2vec, doc):
  '''This function calculates the document embedding from the relevant word
  embeddings using word2vec trained on Google News.
  Inputs : 
  - one document - list of words that make up that document
  Returns :
  - numpy array
  '''

  # Remove OOV words
  doc = [word for word in doc if word in word2vec.vocab]

  # Calculate mean
  return np.mean(word2vec[doc], axis=0)

In [ ]:
test = utils.simple_preprocess('hello there, someone?')
#get_doc_embedding(wv, test)

In [ ]:
num_docs = 767968
num_dims = 300
not_valid = []
to_save = np.empty((num_dims, num_docs))
for i, doc in enumerate(sentences):
  try:
    to_save[:,i] = get_doc_embedding(wv, doc)
  except:
    not_valid.append(i)
    to_save[:,i] = np.zeros(num_dims)

In [ ]:
np.save('word2vec_pretrained.npy', to_save)

In [ ]:
np.save('word2vec_pretrained.npy', vecs)

# Check Embeddings

In [ ]:
vecs = np.load('word2vec_pretrained.npy')

In [ ]:
vecs.shape

(767968, 300)

In [ ]:
num_zeros = 0

for vec in vecs:
  if np.all(vec == 0):
    num_zeros += 1

print(num_zeros)

1


This means that there was 1 paper which did not have any words in the pretrained word2vec's vocabulary. I'm happy to keep this in as it will only affect 1 or a few comparisons out of 10,000.